# Business Insights & Final Recommendations

В этом ноутбуке подводятся итоги проекта Recommendation System
на датасете RetailRocket с точки зрения продуктового и бизнес-применения.

Цель:
- интерпретировать полученные результаты;
- определить, какие модели целесообразно использовать в реальной системе;
- обозначить ограничения и возможные улучшения.


## Исходная бизнес-задача

Дано:
- события пользователей e-commerce платформы (views, add-to-cart, transactions);
- крайне разреженные данные: большинство пользователей совершают 1–3 действия;
- большой каталог товаров и короткие пользовательские сессии.

Бизнес-цель:
- рекомендовать пользователям товары, которые с наибольшей вероятностью
  приведут к покупке;
- максимизировать конверсию при ограниченном числе рекомендаций (Top-N).


## Рассмотренные подходы

В рамках проекта были реализованы и сравнены следующие подходы:

1. **Random baseline**  
   Используется как нижняя граница качества.

2. **Popularity baseline**  
   Рекомендация глобально самых популярных товаров.

3. **Item-based Collaborative Filtering**  
   Рекомендации на основе совместных покупок товаров.

4. **Matrix Factorization (ALS, implicit feedback)**  
   Классическая CF-модель для неявной обратной связи.

5. **Feature-based Ranking (Learning to Rank)**  
   Модель ранжирования с использованием признаков пользователя, товара
   и истории взаимодействий.


## Интерпретация результатов

Сравнение моделей показало:

- Random baseline ожидаемо даёт почти нулевые значения метрик;
- Popularity baseline оказался сильным и стабильным решением,
  что типично для разреженных e-commerce данных;
- Item-based CF и ALS не дали прироста над популярностью в текущей постановке,
  что связано с:
  - малым числом покупок на пользователя;
  - сильным bias в сторону популярных товаров;
  - отсутствием богатых пользовательских историй;
- Feature-based ranking значительно превзошёл все предыдущие подходы
  по `precision@k` и `recall@k`.


## Почему feature-based ranking работает лучше

Ключевая причина успеха ranking-модели — использование явных признаков:

- глобальная популярность товара;
- активность пользователя;
- история конкретной пары (user, item).

В отличие от классических CF-моделей, ранкер:

- не пытается восстановить скрытые латентные факторы
  из крайне разреженных данных;
- напрямую учится предсказывать вероятность покупки;
- гибко комбинирует разные источники сигнала.

Фактически, модель повторяет подход,
используемый в production рекомендательных системах.


## Рекомендации для production

На основе полученных результатов оптимальная архитектура
рекомендательной системы может выглядеть следующим образом:

1. **Candidate generation**
   - popularity-based рекомендации;
   - (опционально) item-based CF для разнообразия.

2. **Ranking**
   - feature-based модель (например, RandomForest / Gradient Boosting),
     обученная на истории взаимодействий.

3. **Cold-start**
   - для новых пользователей — popularity;
   - для новых товаров — ограниченное продвижение + сбор сигналов.

Такой двухэтапный pipeline позволяет:
- масштабироваться на большие каталоги;
- эффективно работать с разреженными данными;
- улучшать качество рекомендаций без сложных моделей.


## Ограничения и возможные улучшения

Ограничения текущего проекта:

- отсутствие сессионных признаков;
- нет информации о категориях и атрибутах товаров;
- ограниченный горизонт истории пользователей.

Потенциальные улучшения:

- добавление временных признаков (recency, frequency);
- переход на более сильные ранкеры (XGBoost, LightGBM);
- использование нейронных моделей для генерации кандидатов;
- онлайн-метрики и A/B тестирование.


## Итог

В рамках проекта был построен полноценный applied ML pipeline
для рекомендательной системы:

- проведён EDA реальных e-commerce данных;
- реализованы базовые и CF-подходы;
- показаны ограничения классических моделей;
- реализован feature-based ranking,
  показавший наилучшее качество.

Проект демонстрирует практический подход
к построению recommendation system
и может служить основой для production-решений.
